In [5]:
import requests
import pickle

EIA_KEY = '7eb8f2869a253fc97eeaefee4154550a'

In [6]:
# get catalog of available electricity time series
# API docs: https://www.eia.gov/opendata/qb.php?category=2122628
catalog_query = 'http://api.eia.gov/category/?api_key={}&category_id=2122628'.format(EIA_KEY)
catalog_response = requests.get(catalog_query).json()

In [7]:
def region_filter(region_pair):
    # return all region-level series, but not the 'US total' series
    if ('region' in region_pair[0]) and ('48' not in region_pair[0]):
        return True
    elif region_pair[0] in ['Demand for Tennessee Valley Authority (TVA), Hourly',
                           'Demand for Electric Reliability Council of Texas, Inc. (ERCO), Hourly',
                           'Demand for New England ISO (ISNE), Hourly',
                           'Demand for New York Independent System Operator (NYIS), Hourly']:
        return True
    else:
        return False

In [8]:
# consider only the region-level time series
# (not going to balancing authority granularity level)
# build an API endpoint URL for each region
region_ids = filter(region_filter,
                    map(lambda region: (region['name'], region['series_id']),
                        catalog_response['category']['childseries']))

In [9]:
region_ids

[(u'Demand for California (region), Hourly', u'EBA.CAL-ALL.D.H'),
 (u'Demand for Carolinas (region), Hourly', u'EBA.CAR-ALL.D.H'),
 (u'Demand for Central (region), Hourly', u'EBA.CEN-ALL.D.H'),
 (u'Demand for Electric Reliability Council of Texas, Inc. (ERCO), Hourly',
  u'EBA.ERCO-ALL.D.H'),
 (u'Demand for Florida (region), Hourly', u'EBA.FLA-ALL.D.H'),
 (u'Demand for New England ISO (ISNE), Hourly', u'EBA.ISNE-ALL.D.H'),
 (u'Demand for Mid-Atlantic (region), Hourly', u'EBA.MIDA-ALL.D.H'),
 (u'Demand for Midwest (region), Hourly', u'EBA.MIDW-ALL.D.H'),
 (u'Demand for Northwest (region), Hourly', u'EBA.NW-ALL.D.H'),
 (u'Demand for New York Independent System Operator (NYIS), Hourly',
  u'EBA.NYIS-ALL.D.H'),
 (u'Demand for Southeast (region), Hourly', u'EBA.SE-ALL.D.H'),
 (u'Demand for Southwest (region), Hourly', u'EBA.SW-ALL.D.H'),
 (u'Demand for Tennessee Valley Authority (TVA), Hourly', u'EBA.TVA-ALL.D.H')]

southern = southeast

In [10]:
# get time series data for each region
region_series = {}
for region in region_ids:
    name = region[0]
    series_id = region[1]
    # API docs: https://www.eia.gov/opendata/qb.php?category=2122628&sdid=EBA.CAL-ALL.D.H
    series_query = 'http://api.eia.gov/series/?api_key={}&series_id={}'.format(EIA_KEY, series_id)
    series_reponse = requests.get(series_query).json()
    series_data = series_reponse['series'][0]['data']
    region_series[name] = series_data

In [12]:
with open('./data/power_series.pickle', 'wb') as handle:
    pickle.dump(region_series, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
#with open('power_series.pickle', 'rb') as handle:
#    region_series = pickle.load(handle)